In [ ]:
%matplotlib widget
import warnings
warnings.filterwarnings("ignore") 

In [ ]:
import time
import numpy as np
from matplotlib import pyplot as plt
from qualang_tools.units import unit
u = unit(coerce_to_integer=True)
from qm.qua import *
from qm import QuantumMachinesManager
from qualang_tools.results import progress_counter, fetching_tool
from qualang_tools.loops import from_array
from progress import addjob, ProgressPlot
import threading
import config_qubit as config

In [ ]:
# Connect to the cluster (run only once)
import QM_cluster
qmm = QuantumMachinesManager(host=QM_cluster.QM_Router_IP, cluster_name=QM_cluster.cluster_name)

# Connect to the running QM

In [ ]:
# Get the## QM reference (rerun every time the config is changed)
qm_list =  qmm.list_open_qms()
qm = qmm.get_qm(qm_list[0])
print(f"Connected to {qm.id}")

# Histogram Analysis

In [ ]:
def rot(I,Q,theta,scale=1e4):
    theta = theta*np.pi/180
    return scale*(I*np.cos(theta)-Q*np.sin(theta)) , scale*(I*np.sin(theta)+Q*np.cos(theta))

## Without parametric amplifier

In [ ]:
with np.load('JPA_off.npz') as data:
    I0 = data['I0']
    I1 = data['I1']
    Q0 = data['Q0']
    Q1 = data['Q1']

- Find the best rotation angle to encode the measurement result on the $I$ quadrature
- Estimate the qubit temperature
- Estimate the readout error

In [ ]:
I0r,Q0r = rot(I0,Q0,0)
I1r,Q1r = rot(I1,Q1,0)
fig,ax = plt.subplots()
ax.plot(I0r, Q0r,',')
ax.plot(I1r, Q1r,',')
ax.set_xlabel("I")
ax.set_ylabel("Q")
ax.set_aspect("equal")

In [ ]:
fig,ax=plt.subplots()
bins = np.arange(-20,20,0.1)
ax.hist(I0r,bins,histtype='step')
ax.hist(I1r,bins,histtype='step');

## With parametric amplifier

In [ ]:
with np.load('JPA_on.npz') as data:
    I0 = data['I0']
    I1 = data['I1']
    Q0 = data['Q0']
    Q1 = data['Q1']

- Find the best rotation angle to encode the measurement result on the $I$ quadrature
- Estimate the qubit temperature
- Estimate the readout error

In [ ]:
I0r,Q0r = rot(I0,Q0,0)
I1r,Q1r = rot(I1,Q1,0)
fig,ax = plt.subplots()
ax.plot(I0r, Q0r,',')
ax.plot(I1r, Q1r,',')
ax.set_xlabel("I")
ax.set_ylabel("Q")
ax.set_aspect("equal")

In [ ]:
fig,ax=plt.subplots()
bins = np.arange(-80,80,0.3)
ax.hist(I0r,bins,histtype='step')
ax.hist(I1r,bins,histtype='step');

# Quantum trajectories

In [ ]:
# Parameters Definition
n_meas = 5000  
###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)
results = fetching_tool(job, data_list=["I", "Q"])
I,Q=results.fetch_all()
print("Done")

In [ ]:
theta = -292*np.pi/180
Ir =  I*np.cos(theta)-Q*np.sin(theta)
Ir *= 1e3
fig,ax=plt.subplots(figsize=(12,4))
ax.plot(Ir)
ax.plot(Ir>.8)

- What is the probability to obtain identical results two times in a row?
- What is the probability to obtain different results two times in a row?
- What is the jump probability from $|0\rangle$ to $|1\rangle$?
- What is the jump probability from $|1\rangle$ to $|0\rangle$?

# Quantum Random Number Generator

In [ ]:
# Parameters Definition
n_meas = 5000  
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        play("pi_half","qubit")
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)
        wait(thermalization_time*u.us)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)
results = fetching_tool(job, data_list=["I", "Q"])
I,Q=results.fetch_all()
print("Done")

In [ ]:
theta = -292*np.pi/180
Ir =  I*np.cos(theta)-Q*np.sin(theta)
Ir *= 1e3
fig,ax=plt.subplots(figsize=(12,4))
ax.plot(Ir)
ax.plot(Ir>.8)

- Extract a random number sequence from the data
- Is it biased? Why?

# Quantum Tomography

- The qubit seems to be half the time in $|0\rangle$ and half the time in $|1\rangle$. But it should be in a well defined deterministic quantum state. How can we prove it?
- By adding pulses before the measurement, try to estimate the qubit quantum state.

In [ ]:
# Parameters Definition
n_meas = 5000  
thermalization_time = 80 #in µs

###################
# The QUA program #
###################
with program() as qmprog:
    n = declare(int)  
    I = declare(fixed)  
    Q = declare(fixed)  
    I_st = declare_stream()  # Stream for the 'I' quadrature
    Q_st = declare_stream()  # Stream for the 'Q' quadrature

    with for_(n, 0, n < n_meas, n + 1):  # QUA for_ loop for averaging
        play("pi_half","qubit")
        measure(
            "readout",
            "resonator",
            dual_demod.full("cos", "sin", I),
            dual_demod.full("minus_sin", "cos", Q),
        )
        save(I, I_st)
        save(Q, Q_st)
        wait(thermalization_time*u.us)

    with stream_processing():
        # Cast the data into a 1D vector, average the 1D vectors together and store the results on the OPX processor
        I_st.save_all("I")
        Q_st.save_all("Q")

# Send the QUA program to the OPX, which compiles and executes it
job = addjob(qmprog, qm)
results = fetching_tool(job, data_list=["I", "Q"])
I,Q=results.fetch_all()
print("Done")

In [ ]:
theta = -292*np.pi/180
Ir =  I*np.cos(theta)-Q*np.sin(theta)
Ir *= 1e3
fig,ax=plt.subplots(figsize=(12,4))
ax.plot(Ir)
ax.plot(Ir>.8)